In [1]:
import pandas as pd
import pyodbc
import urllib

# Installing the SQL Alchemy Algo
from sqlalchemy import event
from sqlalchemy import create_engine

# Specify the connection details
server = '172.16.3.22'
database = 'patentview_trial'
username = 'Tipu_Thakur'
password = 'IDSL2Sb!gt%2'

# Create a connection string
connection_string = f"Driver={{SQL Server}};Server={server};Database={database};UID={username};PWD={password};"

# Establish the database connection
connection = pyodbc.connect(connection_string)



In [2]:
# Create a cursor object
cursor = connection.cursor()

# Execute the first SQL query
query1 = """
SELECT REC_ID,USPTO_PATENT_ID,USPTO_PATENT_TITLE,EMAKG_PAPER_ID,EMAKG_PAPER_TITLE,dcterms_creator,Author_Name,inventor_id,Inventor_Name
FROM [dbo].DERV_TITLE_MAPPING_IDS
"""

cursor.execute(query1)


In [3]:

# Fetch the first query results
results1 = cursor.fetchall()

# Get the column names from the cursor description
columns1 = [column[0] for column in cursor.description]

# Convert the first query results to a DataFrame
df1 = pd.DataFrame.from_records(results1, columns=columns1)


In [4]:
# Define a Jaccard Token Similarity Function
def token_jaccard_similarity(string1, string2):
    set1 = set(string1.split())
    set2 = set(string2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity

In [6]:
df1['token_jaccard_similarity'] = df1.apply(lambda x: token_jaccard_similarity(x['Author_Name'], x['Inventor_Name']), axis=1)

In [7]:
# Export the DataFrame to a file with a delimiter
df1.to_csv('E:\Sagar\EMAKG\Paper_Patent_Linkage\Same_Paper_Name_Jaccard_Scoring.csv', sep='|', index=False)

In [4]:
# Load the DataFrame from File
# df1 = pd.read_csv('E:\Sagar\EMAKG\Paper_Patent_Linkage\Scoring.csv', sep='|')

In [8]:
df1.head()

,REC_ID,USPTO_PATENT_ID,USPTO_PATENT_TITLE,EMAKG_PAPER_ID,EMAKG_PAPER_TITLE,dcterms_creator,Author_Name,inventor_id,Inventor_Name,token_jaccard_similarity
0,2648107,11185864,SAMPLE PREPARATION DEVICE,2727912123,SAMPLE PREPARATION DEVICE,2728974135,È²Å°”Â·È´Å°”Æ ¼É›·Å¾·Å°”,fl:jo_ln:palmerfelgate-2,JOHN PALMER-FELGATE,0.0
1,2648108,11185864,SAMPLE PREPARATION DEVICE,2846357695,SAMPLE PREPARATION DEVICE,2864464661,ZHOU JIWEN,fl:da_ln:haworth-2,DANIEL NICHOLAS HAWORTH,0.0
2,2648109,11185864,SAMPLE PREPARATION DEVICE,2846357695,SAMPLE PREPARATION DEVICE,2864464661,ZHOU JIWEN,fl:jo_ln:palmerfelgate-2,JOHN PALMER-FELGATE,0.0
3,2648110,11185864,SAMPLE PREPARATION DEVICE,2837446964,SAMPLE PREPARATION DEVICE,2869630602,TAJIMA KATSUNORI,fl:da_ln:haworth-2,DANIEL NICHOLAS HAWORTH,0.0
4,2648111,11185864,SAMPLE PREPARATION DEVICE,2837446964,SAMPLE PREPARATION DEVICE,2869630602,TAJIMA KATSUNORI,fl:jo_ln:palmerfelgate-2,JOHN PALMER-FELGATE,0.0


In [9]:
# Loading the File to SQL Table Post Additional Column Altered in SQL DB

params = 'DRIVER='+'ODBC Driver 17 for SQL Server' + ';SERVER='+server + ';PORT=1433;DATABASE=' + database + ';UID=' + username + ';PWD=' + password
cnxn = pyodbc.connect(params)
# Create a cursor object
insert_cursor = cnxn.cursor()

db_params = urllib.parse.quote_plus(params)

engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))

df1.to_sql("DERV_TITLE_MAPPING_IDS", engine, if_exists='append', index=False, schema="dbo")


-1